In [1]:
# James Elgy - 10/08/2023

import numpy as np
from matplotlib import pyplot as plt

# Import libaries
import numpy as np
import netgen.meshing as ngmeshing
from ngsolve import *
import scipy.sparse as sp
from netgen.occ import *
import gc
from ngsolve.la import EigenValues_Preconditioner


# Specify order of elements
for Order in range(0, 2):
    # Specify the mesh file
    Object = "OCC_sphere.vol"
    # Loading the object file
    ngmesh = ngmeshing.Mesh(dim=3)
    ngmesh.Load("VolFiles/" + Object)

    sph = Sphere(Pnt(0,0,0), r=1)
    sph.mat('copper')
    box = Box(Pnt(-10,-10,-10), Pnt(10,10,10))
    box.mat('air')
    box.maxh=100
    sph.maxh=1
    geo = Glue([sph, box])
    mesh = Mesh(OCCGeometry(geo).GenerateMesh())


    # Creating the mesh and defining the element types
    # mesh = Mesh("VolFiles/" + Object)
    curve = 4
    mesh.Curve(curve)  # This can be used to set the degree of curved elements
    numelements = mesh.ne  # Count the number elements
    print(" mesh contains " + str(numelements) + " elements")

    # Materials consist of sphere material and air in the order as defined in the mesh
    matlist = ["copper", "air"]
    contrast = 20
    murlist = [contrast, 1]
    inout = []
    for mat in matlist:
        if mat == "air":
            inout.append(0)
        else:
            inout.append(1)
    inorout = dict(zip(matlist, inout))
    murmat = dict(zip(matlist, murlist))

    # Coefficient functions
    mur_coef = [murmat[mat] for mat in mesh.GetMaterials()]
    mur = CoefficientFunction(mur_coef)
    # inout = 1 in B, inout =0 in R^3 \ B
    inout_coef = [inorout[mat] for mat in mesh.GetMaterials()]
    inout = CoefficientFunction(inout_coef)

    # define material constants
    Mu0 = 4. * np.pi * 1e-7
    Epsilon = 1e-6

    # Coefficent function for background field
    B0 = 1
    H0 = CoefficientFunction((0, 0, B0 / Mu0))
    alpha = 1
    exactsphere = 1

    # define tolerances etc
    Maxsteps = 500
    Tolerance = 1e-8
    Solver = "mult"

    print("Order=", Order)

    dom_nrs_metal = [0 if mat == "air" else 1 for mat in mesh.GetMaterials()]
    fes = HCurl(mesh, order=Order, dirichlet="outer", complex=False, gradientdomains=dom_nrs_metal)

    # Count the number of degrees of freedom
    ndof = fes.ndof
    print("ndof", ndof)

    # Set up the grid function and apply Dirichlet BC
    a = GridFunction(fes)

    # Setup boundary condition
    if exactsphere == 0:
        a.Set((0, 0, 0), BND)
    else:
        def axout(x, y, z):
            r = sqrt(x ** 2 + y ** 2 + z ** 2);
            theta = acos(z / r);
            phi = atan2(y, x);
            Aphi = 0.5 * B0 * (r - 2 * (1 - contrast) / (2 + contrast) * alpha ** 3 / r ** 2) * sin(theta);
            return -sin(phi) * Aphi


        def ayout(x, y, z):
            r = sqrt(x ** 2 + y ** 2 + z ** 2);
            theta = acos(z / r);
            phi = atan2(y, x);
            Aphi = 0.5 * B0 * (r - 2 * (1 - contrast) / (2 + contrast) * alpha ** 3 / r ** 2) * sin(theta);
            return cos(phi) * Aphi


        def azout(x, y, z):
            return 0.


        a.Set((axout(x, y, z), ayout(x, y, z), azout(x, y, z)), BND)

    # Setup source condition
    src = CoefficientFunction((0, 0, 0))

    # Test and trial functions
    u = fes.TrialFunction()
    v = fes.TestFunction()

    Additional_Int_Order = 2
    # Create the linear and bilinear forms (nb we get the linear system TSM.mat a.vec.data = - R = f.vec.data)
    f = LinearForm(fes)
    f += SymbolicLFI((src * v), bonus_intorder=Additional_Int_Order)

    TSM = BilinearForm(fes, symmetric=True, condense=True)
    TSM += SymbolicBFI(inout * (mur ** (-1)) * (curl(u) * curl(v)), bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI((1 - inout) * (curl(u) * curl(v)), bonus_intorder=Additional_Int_Order)
    TSM += SymbolicBFI(Epsilon * (u * v), bonus_intorder=Additional_Int_Order)






 # Setting preconditioner.

    if Solver == 'mult':
        TSM.Assemble()
        f.Assemble()
        from wave_propagation.Preconditioners.multigrid import MGPreconditioner
        pre = MGPreconditioner(fes, 0, TSM.mat, None, True, finest_level=0)
        for l in range(3):

            # Only refining inside sphere:
            for el in mesh.Elements():
                mesh.SetRefinementFlag(el, el.mat == 'copper')

            mesh.Refine()
            print(f'NE = {mesh.ne}')
            fes.Update()
            a.Update()
            TSM.Assemble()
            f.Assemble()

            pre = MGPreconditioner(fes, l+1, TSM.mat, pre, True, finest_level=0)

            lam = EigenValues_Preconditioner(TSM.mat, pre)
            print("ndof=%7d:  minew=%.4f  maxew=%1.4f  Cond# = %5.3f"
                  % (fes.ndof, lam[0], lam[-1], lam[-1] / lam[0]))

        P = pre

importing NGSolve-6.2.2204
 mesh contains 3842 elements
Order= 0
ndof 4608


ModuleNotFoundError: No module named 'wave_propagation'